此ipynb相较mlhw5.ipynb是为了验证我对chat template实现而写的

# 模型

In [1]:
model_path = "/home/yxlin/huggingface/LLaMA-2-7b-bnb-4bit"

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048   # 官方教程推荐
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_path,    ### Do not change the model for any other models or quantization versions
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, ### TODO : Choose any number > 0 ! Common values are 4, 8, 16, 32, 64, 128. Higher ranks allow more expressive power but also increase parameter count.
    lora_alpha = 16,  ### TODO : Choose any number > 0 ! Suggested 4, 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/yxlin/github/LHY_ML2025/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.516 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 数据

In [14]:
from datasets import load_dataset

data_path = "/home/yxlin/github/LHY_ML2025/MLHW5/dataset/data/"
data_name = "train-00000-of-00001-a09b74b3ef9c3b56.parquet"

dataset = load_dataset(
    data_path,
    data_files=data_name
)

print(dataset)

from datasets import load_dataset, Dataset

def compute_conversation_length(message):
    # 增加一个简单的安全检查，防止 input 为 None
    instruction = message['instruction'] or ""
    input_text = message['input'] or ""
    output = message['output'] or ""
    return len(instruction) + len(input_text) + len(output)

# 修改这里：使用 dataset['train']
sorted_dataset_list = sorted(dataset['train'], key=compute_conversation_length, reverse=False)

# 注意：sorted_dataset_list 现在是一个 Python List（列表），不再是 HuggingFace Dataset 对象
print(sorted_dataset_list[0])

sorted_dataset_list = sorted_dataset_list[:100]
print("\nTop examples sorted by simple conversation length:")
for id, entry in enumerate(sorted_dataset_list[:5]):
    print(f"ID: {id}, Conversation Length: {compute_conversation_length(entry)}")

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})
{'instruction': 'Solve 8 x 8.', 'input': '', 'output': '64', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSolve 8 x 8.\n\n### Response:\n64'}

Top examples sorted by simple conversation length:
ID: 0, Conversation Length: 14
ID: 1, Conversation Length: 14
ID: 2, Conversation Length: 15
ID: 3, Conversation Length: 15
ID: 4, Conversation Length: 15


## format 数据

将数据组织成可以被`tokenizer.apply_chat_template(messages, tokenize=False)`处理的形式
我们的数据为messages，其交给`tokenizer.apply_chat_template`处理，tokenizer会依据自身设置的模板从messages中找数据填补进行，形成最终喂给模型训练的标准的数据

messages是个list, 其中元素为dict:
```
messages = [
    {"role": "system", "content": "你是一个严谨的数学家。"},
    {"role": "user", "content": "1 + 1 等于几？"},
    {"role": "assistant", "content": "2"},
    {"role": "system", "content": "你是一个严谨的语言家。"},
    {"role": "user", "content": "1 + 1 是什么意思？"},
    {"role": "assistant", "content": "数学上的数值1两两相加"},
    ...
]
```
交给`tokenizer.apply_chat_template`处理后我们可以得到如下类似输出，取决于tokenizer的模板：
```
<|im_start|>system\n你是一个严谨的数学家。<|im_end|>\n<|im_start|>user\n1 + 1 等于几？<|im_end|>\n<|im_start|>assistant\n2<|im_end|>\n<|im_start|>system\n你是一个严谨的语言家。<|im_end|>\n<|im_start|>user\n1 + 1 等于几？<|im_end|>\n<|im_start|>assistant\n数学上的数值1两两相加<|im_end|>
```

In [15]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",  ### Use llama-3.1 template for better performance here
)

# 查看现在的模板是什么样子的
print(tokenizer.chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 July 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content'] %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>

" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython
" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "

"}}
{%- endif %}
{{- "

In [16]:
def format_message(message: dict, system_prompt: str) -> list[dict]:
    '''针对使用dataset读取表格文件后进行format'''
    chats = []

    chats.append(
        {
            "role": "system",
            "content": f"{system_prompt}"
        }
    )
    chats.append(
        {
            "role": "user",
            "content": f"{message['instruction']}\n{message['input']}"
        }
    )
    chats.append(
        {
            "role": "assistant",
            "content": f"{message['output']}"
        }
    )
    return chats

import re

def remove_meta_info(text: str) -> str:
    pattern = r"Cutting Knowledge Date:.*?\nToday Date:.*?\n\n"
    return re.sub(pattern, "", text, flags=re.DOTALL)

system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."
train_dataset = []
for message in sorted_dataset_list:
    chats = format_message(message=message, system_prompt=system_prompt)
    train_sample = tokenizer.apply_chat_template(chats, tokenize=False)     # train_sample是个str类型
    train_sample = remove_meta_info(train_sample)   # 删除掉模板中附带的时间信息
    train_dataset.append(
        {
            "text": train_sample
        }
    )

train_dataset = Dataset.from_list(train_dataset)

In [17]:
import random

for i in range(3):
    idx = random.randint(0, len(train_dataset) - 1)
    print(f"--- 样本 {idx} ---")
    print(train_dataset[idx]["text"])
    print("\n" + "="*50 + "\n")

--- 样本 42 ---
<s><|start_header_id|>system<|end_header_id|>

You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>

Add 50 to this number.
230<|eot_id|><|start_header_id|>assistant<|end_header_id|>

280<|eot_id|>


--- 样本 27 ---
<s><|start_header_id|>system<|end_header_id|>

You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>

Divide the number 8 by 2.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

4<|eot_id|>


--- 样本 51 ---
<s><|start_header_id|>system<|end_header_id|>

You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>

Label the given input.
dog<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Noun

这里很诡异，没有结束符</s>，这本来是模板负责的，但是这么并没有生成。

# 训练

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


################# TODO : Tweak the training hyperparameters here.  #####################


training_config = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "warmup_steps": 10,
    "num_train_epochs": 2,
    "learning_rate": 2e-4,
    "optim": "adamw_8bit",
    "weight_decay": 0.01,
    "lr_scheduler_type": "linear",
    "seed": 3407,   ### Do not modify the seed for reproducibility
}


################# TODO #################################################################

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = training_config["per_device_train_batch_size"],
        gradient_accumulation_steps = training_config["gradient_accumulation_steps"],
        warmup_steps = training_config["warmup_steps"],
        num_train_epochs = training_config["num_train_epochs"], # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = training_config["learning_rate"],
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = training_config["optim"],
        weight_decay = training_config["weight_decay"],
        lr_scheduler_type = training_config["lr_scheduler_type"],
        seed = training_config["seed"],
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=84): 100%|██████████| 100/100 [00:06<00:00, 15.90 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 2 | Total steps = 14
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Step,Training Loss
1,2.187700
2,2.164100
3,2.170700
4,2.152500
5,2.124700
6,2.056700
7,1.900700
8,1.751600
9,1.501400
10,1.256200


# 推理

In [19]:
# 1. 切换到推理模式（Unsloth 专用加速）
model = FastLanguageModel.for_inference(model)

In [20]:
system_prompt = "You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request."
input_text = "你好, 今天天气不错。所以今天星期几？" 
text = [
    {
        "role": "system",
        "content": system_prompt,
    },
    {
        "role": "user",
        "content": input_text,
    }
]

text = tokenizer.apply_chat_template(text, tokenize=False)     # train_sample是个str类型
text = remove_meta_info(text)   # 删除掉模板中附带的时间信息

print(text)


<s><|start_header_id|>system<|end_header_id|>

You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>

你好, 今天天气不错。所以今天星期几？<|eot_id|>


In [21]:
# 3. 编码并生成
inputs = tokenizer([text], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 512)

# 4. 解码输出
response = tokenizer.batch_decode(outputs)
print(response[0])

<s><s> <|start_header_id|>system<|end_header_id|>

You are a AI Helper. Below is an instruction that describes a task. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>

你好, 今天天气不错。所以今天星期几？<|eot_id|>

星期五<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

星期五<|eot_id|>

<|start_header_id|>user<|end_header_id|>

恭喜你，你说的对吧。<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

恭喜你，你说的对吧。<|eot_id|>

<|start_header_id|>user<|end_header_id|>

我很高兴你说的对吧。<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

我很高兴你说的对吧。<|eot_id|>

<|start_header_id|>user<|end_header_id|>

谢谢你。<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

你好。<|eot_id|>

<|start_header_id|>user<|end_header_id|>

你好。<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

你好，我是杨。<|eot_id|>

<|start_header_id|>user<|end_header_id|>

你好，我是杨。<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>

你好，我是杨。<|eot_id|>

<|start_header_id|>user<|en

**我可以如何理解tokenizer的chat_template? 我有一份chat_template，我应该如何理解他？**

你可以把 `chat_template` 理解为一种**“翻译协议”**。

在与大语言模型（LLM）对话时，我们习惯用“用户”和“助手”的身份进行交流。但模型本身本质上只是一个“文本补全器”，它并不知道什么是对话。它需要特定的**特殊标记（Special Tokens）**来区分：哪里是你的指令，哪里是它的回答，哪里又是系统设定。

`chat_template` 的作用就是：**将你的结构化对话历史（List of Dicts），自动转换成模型能听懂的纯文本字符串。**

---

### 1. 核心逻辑：从 JSON 到 String

通常你的对话数据长这样：

```python
messages = [
    {"role": "system", "content": "你是一个严谨的数学家。"},
    {"role": "user", "content": "1 + 1 等于几？"}
]

```

如果没有 `chat_template`，你得手动拼接字符串（比如加 `\n` 或 `User:`）。但不同的模型（如 Llama 3, Qwen, DeepSeek）要求的格式完全不同。有了它，你只需要调用：
`tokenizer.apply_chat_template(messages, tokenize=False)`

它就会根据模板输出类似：
`<|im_start|>system\n你是一个严谨的数学家。<|im_end|>\n<|im_start|>user\n1 + 1 等于几？<|im_end|>\n<|im_start|>assistant\n`

---

### 2. 如何读懂一份 `chat_template` 代码？

大部分 `chat_template` 是用 **Jinja2** 语法编写的。你可以关注以下几个关键点：

#### A. 循环结构 (`{% for message in messages %}`)

这是模板的核心。它会遍历你提供的对话列表，逐条处理。

#### B. 角色判定 (`{% if message['role'] == 'user' %}`)

模板会检查 `role` 是 `system`、`user` 还是 `assistant`，并为不同角色包裹不同的**控制符**。

* **System:** 通常放在开头，定义边界。
* **User:** 提示模型这是输入的指令。
* **Assistant:** 提示模型这是它该生成的回复。

#### C. 特殊标记 (BoS/EoS)

你会看到类似 `{{ bos_token }}` 或 `<|im_start|>` 这种符号：

* **BoS (Beginning of Sequence):** 告诉模型，“对话从这里开始”。
* **EoS (End of Sequence):** 告诉模型，“这条消息结束了，你可以开始回答或停止了”。

#### D. 空白符处理 (`-` 符号)

如果你看到 `{%-` 或 `-%}`，这代表**去掉多余的空格或换行符**，确保拼接出来的字符串非常紧凑，不浪费 Token。

---

### 3. 为什么它如此重要？

1. **防止“幻觉”和“角色错乱”：** 如果格式不对，模型可能会分不清到底是谁在说话，甚至直接续写你的问题而不是回答。
2. **训练与推理的一致性：** 模型在训练时用的是什么格式，我们在使用（推理）时就必须严格保持一致，否则效果会大打折扣。
3. **自动添加生成引导：** 好的模板会在最后自动加上 `assistant\n` 之类的后缀，像接力棒一样交给模型，让它知道：“该你说话了”。

---

### 一个典型的例子 (以 Qwen/ChatML 为例)

```jinja2
{% for message in messages %}
    {{ '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n' }}
{% endfor %}
{% if add_generation_prompt %}
    {{ '<|im_start|>assistant\n' }}
{% endif %}

```

* **前三行：** 把每条消息包装在 `<|im_start|>` 和 `<|im_end|>` 之间。
* **最后三行：** 如果设置了 `add_generation_prompt=True`，它会在末尾强行塞入一个助手的开头，诱导模型开始生成。

---

**如果你手头正有一份具体的 `chat_template` 代码，可以贴出来，我可以帮你逐行解析它的具体含义。你想让我分析一下吗？**

**{% 和 %}这是什么语法？**

这属于 **Jinja2** 模板引擎的语法。

在 Python 领域，Jinja2 是最流行的模板语言（类似于 Web 开发中将 HTML 和逻辑代码结合的工具）。在 LLM（大语言模型）的 `tokenizer` 中，它被专门用来处理对话格式的拼接。

你可以把 `{% ... %}` 理解为**“逻辑控制块”**，而把 `{{ ... }}` 理解为**“内容打印块”**。

---

### 1. `{% ... %}` ：逻辑控制 (Logic)

这类符号包裹的是**指令**，它不直接输出文本，而是告诉程序该怎么做。

* **条件判断 (If/Else):**
```jinja2
{% if message['role'] == 'user' %}
    (如果是用户，就执行这里的操作)
{% endif %}

```


* **循环 (For Loop):**
```jinja2
{% for message in messages %}
    (针对对话列表里的每一条消息，重复执行操作)
{% endfor %}

```


* **变量赋值:**
```jinja2
{% set loop_messages = messages %}

```



---

### 2. `{{ ... }}` ：内容输出 (Expression)

这类符号包裹的是**变量或表达式**，它会把结果直接“写”进最终生成的字符串里。

* **输出变量:** `{{ bos_token }}` 会被替换成具体的起始符（如 `<s>`）。
* **输出内容:** `{{ message['content'] }}` 会被替换成你输入的对话文字。
* **字符串拼接:** `{{ '<|user|>\n' + message['content'] }}`。

---

### 3. 特殊符号：`{%-` 和 `-%}` (去空格)

在 `chat_template` 中，你经常会看到里面多了一个短横线。这非常关键：

* **作用：** 自动去掉该块之前或之后的**空白字符**（空格、制表符、换行符）。
* **原因：** LLM 对空格非常敏感。如果你在编写模板时为了美观换了行，但不想让这些换行符进入最终的 Token 中，就必须用 `{%-`。

---

### 总结对比

| 符号 | 名称 | 作用 | 例子 |
| --- | --- | --- | --- |
| `{% ... %}` | 控制块 | 执行**逻辑**（判断、循环） | `{% for m in messages %}` |
| `{{ ... }}` | 变量块 | 打印**内容**（变量、文字） | `{{ m['content'] }}` |
| `{# ... #}` | 注释块 | 内部代码**注释**，不运行 | `{# 这是注释 #}` |

**简单理解：**

* `{% %}` 是**大脑**，负责思考逻辑（如果...那么...）。
* `{{ }}` 是**嘴巴**，负责把内容大声说出来（填入字符串）。

**你想试着把你的那份 `chat_template` 贴出来，让我们看看它具体用了哪些逻辑吗？**